<a href="https://colab.research.google.com/github/GregB77/calor_run/blob/main/work_on_gpx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CALCUL DU NOMBRE DE POINTS DU TRACE GPX OU GEOJSON

In [ ]:
import json
import xml.etree.ElementTree as ET
import os

def compter_points_geospatial(nom_fichier):
    """
    Détermine si le fichier est un GPX ou GeoJSON et compte le nombre total de points.

    Args:
        nom_fichier (str): Le chemin d'accès au fichier (.gpx ou .geojson).

    Returns:
        int: Le nombre total de points, ou -1 si le format n'est pas supporté.
    """
    if not os.path.exists(nom_fichier):
        print(f"Erreur : Le fichier '{nom_fichier}' est introuvable.")
        return 0

    extension = nom_fichier.lower().split('.')[-1]

    # ------------------------------------
    # Traitement GPX
    # ------------------------------------
    if extension == 'gpx':
        try:
            # Namespaces pour trouver les balises <trkpt>
            ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}

            tree = ET.parse(nom_fichier)
            root = tree.getroot()

            # Compte tous les points de trace (trkpt) et de route (rtept)
            points_trace = root.findall('.//gpx:trkpt', ns)
            points_route = root.findall('.//gpx:rtept', ns)

            total_points = len(points_trace) + len(points_route)

            print(f"Format détecté : GPX (Généralement des points de trace ou de route)")
            return total_points

        except ET.ParseError:
            print(f"Erreur de parsing : Le fichier GPX '{nom_fichier}' semble mal formé.")
            return -1
        except Exception as e:
            print(f"Erreur lors du traitement du fichier GPX : {e}")
            return -1

    # ------------------------------------
    # Traitement GeoJSON
    # ------------------------------------
    elif extension == 'geojson' or extension == 'json':
        try:
            with open(nom_fichier, 'r', encoding='utf-8') as f:
                data = json.load(f)

            total_points = 0

            # Fonction récursive pour extraire toutes les coordonnées
            def extraire_coordonnees(geometry):
                if isinstance(geometry, list):
                    # Cas d'une liste de coordonnées (ex: LineString, MultiPoint)
                    # Si c'est un point, c'est une liste de deux/trois nombres (lat, lon, [alt])
                    if all(isinstance(coord, (int, float)) for coord in geometry) and len(geometry) >= 2:
                        return 1
                    # Sinon, on itère sur les sous-listes (ex: anneau dans Polygon, point dans MultiLineString)
                    else:
                        count = 0
                        for item in geometry:
                            count += extraire_coordonnees(item)
                        return count
                return 0

            # Itération sur toutes les 'Features' dans le FeatureCollection
            if data.get('type') == 'FeatureCollection':
                for feature in data.get('features', []):
                    geom = feature.get('geometry')
                    if geom and 'coordinates' in geom:
                        total_points += extraire_coordonnees(geom['coordinates'])

            # Cas d'une seule Feature
            elif data.get('type') == 'Feature':
                geom = data.get('geometry')
                if geom and 'coordinates' in geom:
                    total_points += extraire_coordonnees(geom['coordinates'])

            # Cas d'un simple objet géométrique (ex: LineString direct)
            elif 'coordinates' in data and 'type' in data:
                total_points += extraire_coordonnees(data['coordinates'])

            print(f"Format détecté : GeoJSON (Peut inclure LineString, Polygon, MultiPoint, etc.)")
            return total_points

        except json.JSONDecodeError:
            print(f"Erreur de décodage : Le fichier GeoJSON '{nom_fichier}' est mal formé.")
            return -1
        except Exception as e:
            print(f"Erreur lors du traitement du fichier GeoJSON : {e}")
            return -1

    # ------------------------------------
    # Format non supporté
    # ------------------------------------
    else:
        print(f"Erreur : Extension de fichier non supportée : .{extension}")
        return -1

# =========================================================
# EXEMPLE D'UTILISATION
# =========================================================

# 1. Utilisation avec votre fichier GPX :
gpx_file = "Calor Run 10km.gpx"
nb_points_gpx = compter_points_geospatial(gpx_file)
print(f"Le fichier '{gpx_file}' contient {nb_points_gpx} points.")

# 2. Utilisation avec un fichier GeoJSON (assumant que vous avez créé celui-ci) :
# Attention: Remplacez 'chemin_vers_votre_fichier.geojson' par le nom du fichier que vous souhaitez tester.
# geojson_file = "Calor_Run_10km_full.geojson"
# nb_points_geojson = compter_points_geospatial(geojson_file)
# print(f"Le fichier '{geojson_file}' contient {nb_points_geojson} points.")

Erreur : Le fichier 'Calor Run 10km.gpx' est introuvable.
Le fichier 'Calor Run 10km.gpx' contient 0 points.


GPX TO GEOJSON

In [ ]:
import xml.etree.ElementTree as ET
import json

input_file = "Calor Run 10km.gpx"
output_file = "Calor_Run_10km.geojson"

# Initialisation du GeoJSON
geojson = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "properties": {
                "name": "Unknown",
                "coordinateProperties": [] # Stockera les infos highway/surface
            },
            "geometry": {
                "type": "LineString",
                "coordinates": []
            }
        }
    ]
}

tree = ET.parse(input_file)
root = tree.getroot()

# Namespaces (Le namespace par défaut est souvent problématique pour 'find', on le gère ici)
ns = {'gpx': 'http://www.topografix.com/GPX/1/1'}

# 1. Récupération du nom
name = root.find('.//gpx:metadata/gpx:name', ns)
if name is not None:
    geojson['features'][0]['properties']['name'] = name.text

# 2. Parcours des points (Track Points)
for trkpt in root.findall('.//gpx:trkpt', ns):
    # -- Géométrie --
    lat = float(trkpt.get('lat'))
    lon = float(trkpt.get('lon'))
    ele_tag = trkpt.find('gpx:ele', ns)
    ele = float(ele_tag.text) if ele_tag is not None else 0

    geojson['features'][0]['geometry']['coordinates'].append([lon, lat, ele])

    # -- Extensions (highway / surface) --
    # On initialise les valeurs par défaut
    props = {"highway": None, "surface": None}

    extensions = trkpt.find('gpx:extensions', ns)
    if extensions is not None:
        # Note: Dans votre fichier, <type> semble être direct ou via un namespace enfant
        # On cherche la balise 'type' peu importe le namespace pour être robuste
        type_node = extensions.find('type')
        if type_node is None:
             # Si le XML utilise un namespace pour 'type' non déclaré dans root
             for child in extensions:
                 if child.tag.endswith('type'):
                     type_node = child
                     break

        if type_node is not None:
            highway = type_node.find('highway')
            surface = type_node.find('surface')

            if highway is not None: props['highway'] = highway.text
            if surface is not None: props['surface'] = surface.text

    # Ajout des propriétés spécifiques à ce point dans la liste
    geojson['features'][0]['properties']['coordinateProperties'].append(props)

# 3. Sauvegarde
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(geojson, f, indent=2, ensure_ascii=False)

print(f"Fichier généré : {output_file} avec {len(geojson['features'][0]['geometry']['coordinates'])} points traités.")

Fichier généré : Calor_Run_10km.geojson avec 460 points traités.
